In [2]:
%matplotlib inline

import sys
sys.path.append('../python/')

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
import imp
import cdr
import json
import os.path
from scipy.spatial.distance import cosine
from pandas.io.json import json_normalize

imp.reload(cdr)

print ("Libraries loaded")



Libraries loaded


In [7]:
# aaa = pd.read_csv('../../data/CDR/sms-call-internet-mi-2013-11-27.txt', delimiter='\t', header=None) 
# aaa.columns = ["cellId", "time", "countryCode", "smsIn", "smsOut","callIn", "callOut", "internet"]
# print ("Grid file loaded")

# # change miliseconds to datetime
# aaa.index = pd.to_datetime(aaa['time'],unit='ms',utc=True)
# aaa.index = aaa.index.tz_localize('UTC').tz_convert('Europe/Rome')
# aaa['date'] = aaa.index
# aaa['time_hour'] = aaa.index.hour
# aaa['month'] = aaa.index.month
# aaa['day'] = aaa.index.day
# # aaa = aaa.groupby(['cellId','month','day','time_hour'], as_index=False)
# aaa = aaa.groupby(['cellId','month','day','time_hour'], as_index=False).agg({                 
#             'smsIn': 'sum',
#             'smsOut': 'sum',
#             'callIn': 'sum',
#             'callOut': 'sum',
#             'internet': 'sum'
#         })


Grid file loaded


In [8]:
# # aaa.cellId = aaa.cellId.apply(lambda x: str(x).zfill(5))
# aaa.day = aaa.day.apply(lambda x: str(x).zfill(2))
# aaa.time_hour = aaa.time_hour.apply(lambda x: str(x).zfill(2))

# aaa.index = aaa.month.map(str) + aaa.day + aaa.time_hour
# aaa['unique'] = aaa.index
# # aaa.cellId = aaa.cellId.map(int)
# xx = aaa.pivot(index='cellId', columns='unique', values='internet')

In [ ]:
dfs = pd.DataFrame(columns=["cellId", "time", "smsIn", "smsOut","callIn", "callOut", "internet"])

# Read all the files in Milano
for month in {"11","12"}:
    for day in range(1, 32):
        to_read = '../../data/CDR/sms-call-internet-mi-2013-' + month + '-' +\
                    str(day).zfill(2) + '.txt'
        
        if os.path.isfile(to_read):
            temp = pd.read_csv(to_read, delimiter='\t', header=None)
            temp.columns = ["cellId", "time", "countryCode", "smsIn", "smsOut","callIn", "callOut", "internet"]
#             temp = temp[temp['countryCode'] != 0]
            # change miliseconds to datetime
            temp.index = pd.to_datetime(temp['time'],unit='ms',utc=True)
            temp.index = temp.index.tz_localize('UTC').tz_convert('Europe/Rome')
            temp['date'] = temp.index
            temp['time_hour'] = temp.index.hour
            temp['month'] = temp.index.month
            temp['day'] = temp.index.day

            temp = tp.groupby(['cellId','month','day','time_hour'], as_index=False).agg({                            
                        'smsIn': 'sum',
                        'smsOut': 'sum',
                        'callIn': 'sum',
                        'callOut': 'sum',
                        'internet': 'sum'
                    })            

            dfs = dfs.append(temp)
            print ("loaded ", to_read)

print ("Milano files loaded")

In [ ]:

dfs.index = dfs.cellId
dfs.fillna(0, inplace=True)

In [ ]:
# loading the region-cell data
table = pd.read_csv('../../data/CDR/hash/intersect.csv', header = None) 
table.columns = ['region', 'proportions']
table.index = table.region
table.sort_values(['region'], inplace=True)

# loading the cell-proportion data
prop_table = pd.read_csv('../../data/CDR/hash/cell_intersect.csv', header = None) 
prop_table.columns = ['cell', 'proportions']
prop_table.index = prop_table.cell
prop_table.sort_values(['cell'], inplace=True)

def get_cells_per_region(table, region_id):
    ids = table.iloc[region_id].proportions
    ids = ast.literal_eval(table.get_value(region_id, "proportions"))
    return ids.keys()

def get_call_data(region, df):        
    r_dict = get_cells_per_region(table, region)

    subset = df[df.index.isin(r_dict)]
            
    return subset

def calculate_actual_call(cell_id, region_id, call):
    """
        Create another column on the subset DataFrame that is proportional to the regions.
    """
    prop = ast.literal_eval(prop_table.get_value(int(cell_id), "proportions"))

    try:
        final = prop[str(region_id)] * float(call)
    except:
        final = 0
    
    return final

In [ ]:
# dfs['1101'].fillna(0, inplace=True)
# dfs['1101'].columns = ["cellId", "time", "countryCode", "smsIn", "smsOut",
#                    "callIn", "callOut", "internet"]

# dfs['1101'] = dfs['1101'][dfs['1101']['countryCode'] != 0]
# dfs['1101'].index = dfs['1101'].cellId

In [ ]:
dfs.head()

In [ ]:
len(tp)

In [ ]:
tp = dfs

In [ ]:
# change miliseconds to datetime
tp.index = pd.to_datetime(tp['time'],unit='ms',utc=True)
tp.index = tp.index.tz_localize('UTC').tz_convert('Europe/Rome')
tp['date'] = tp.index
tp['time_hour'] = tp.index.hour
tp['month'] = tp.index.month
tp['day'] = tp.index.day

tp = tp.groupby(['cellId','month','day','time_hour'], as_index=False).agg({     
            'time': 'min',
            'smsIn': 'sum',
            'smsOut': 'sum',
            'callIn': 'sum',
            'callOut': 'sum',
            'internet': 'sum'
        })
tp.index = tp.cellId

In [ ]:
pd.options.mode.chained_assignment = None



In [ ]:
region_based = pd.DataFrame(columns=['region_id','time','adjusted_smsIn','adjusted_smsOut','adjusted_callIn','adjusted_callOut','adjusted_internet'])
import ast
import numpy as np
for re in range(1, 86):
    print(re)
    subdf = get_call_data(re-1, tp)
    subdf["region_id"] = re
    try:
        subdf["adjusted_smsIn"] = np.vectorize(calculate_actual_call)(subdf["cellId"], subdf["region_id"], subdf["smsIn"])
        subdf["adjusted_smsOut"] = np.vectorize(calculate_actual_call)(subdf["cellId"], subdf["region_id"], subdf["smsOut"])
        subdf["adjusted_callIn"] = np.vectorize(calculate_actual_call)(subdf["cellId"], subdf["region_id"], subdf["callIn"])
        subdf["adjusted_callOut"] = np.vectorize(calculate_actual_call)(subdf["cellId"], subdf["region_id"], subdf["callOut"])
        subdf["adjusted_internet"] = np.vectorize(calculate_actual_call)(subdf["cellId"], subdf["region_id"], subdf["internet"])
    except:
        print ("error in making a column...")
        continue
        
    # do aggregation for 
    subdf = subdf.groupby("time").agg({
                'region_id': 'first',
                'adjusted_smsIn': 'sum',
                'adjusted_smsOut': 'sum',
                'adjusted_callIn': 'sum',
                'adjusted_callOut': 'sum',
                'adjusted_internet': 'sum'
            })
    
    region_based = region_based.append(subdf)

region_based.time = region_based.index


In [ ]:
internet = region_based.pivot(index='region_id', columns='time', values='adjusted_internet')

In [ ]:
internet.fillna(0, inplace=True)

In [ ]:
internet.head()

In [ ]:
x = internet.as_matrix()

In [ ]:
mean = np.mean(internet, 0)
mean.count()

In [ ]:
x_prime = np.zeros(shape=(85,mean.count()))

In [ ]:
for i in range(0, 85):
    x_prime[i] = x[i] - mean


In [ ]:
x_prime = np.transpose(x_prime)
len(x_prime)
xpp = pd.DataFrame(x_prime)

In [ ]:
xpp = pd.DataFrame(x_prime)
corr = xpp.corr('pearson')
corr.index = corr.index + 1
corr.columns = range(1,86)

In [ ]:
corr.head()

In [ ]:
f = plt.figure(figsize=(12, 20))
gs = plt.GridSpec(15, 1)
map_ax = f.add_subplot(gs[0:-5])
sns.heatmap(corr, ax=map_ax,
            cbar_kws={"orientation": "horizontal"})
map_ax.set(xlabel="Calls");

In [ ]:
callIn = region_based.pivot(index='region_id', columns='time', values='adjusted_callIn')
callIn.fillna(0, inplace=True)
x = callIn.as_matrix()
mean = np.mean(callIn, 0)
x_prime = np.zeros(shape=(85, mean.count()))
for i in range(0, 85):
    x_prime[i] = x[i] - mean
x_prime = np.transpose(x_prime)

xpp = pd.DataFrame(x_prime)
corr = xpp.corr('pearson')
corr.index = corr.index + 1
corr.columns = range(1,86)

f = plt.figure(figsize=(12, 20))
gs = plt.GridSpec(15, 1)
map_ax = f.add_subplot(gs[0:-5])
sns.heatmap(corr, ax=map_ax,
            cbar_kws={"orientation": "horizontal"})
map_ax.set(xlabel="Call-In");